In [ ]:
#import torch packages
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.utils.data as data
import numpy as np
from scipy.io import loadmat,savemat

In [ ]:
#input representations
input_nodes=512
output_nodes=2048
middle_nodes=512
#functional representations
lr=1e-3
dropout_rate=0.05
max_epoch=10
num_BS=4
num_beam=512

In [ ]:
#define a Network
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.fc1 = nn.Linear(input_nodes,middle_nodes)
        self.fc2 = nn.Linear(middle_nodes,middle_nodes)
        self.fc3 = nn.Linear(middle_nodes,middle_nodes)
        self.fc4 = nn.Linear(middle_nodes,middle_nodes)
        self.fc5 = nn.Linear(middle_nodes,output_nodes)
        
    def forward(self,x):
        # 4 hidden layers with each fc + relu + dropout
        x = x.view(-1,512)
        x = F.dropout(F.relu(self.fc1(x)),p =dropout_rate,training = True)
        x = F.dropout(F.relu(self.fc2(x)),p =dropout_rate,training = True)
        x = F.dropout(F.relu(self.fc3(x)),p =dropout_rate,training = True)
        x = F.dropout(F.relu(self.fc4(x)),p =dropout_rate,training = True)
        # 1 output layer with fc + relu
        x = F.relu(self.fc5(x))
        x = x.view(-1,2048)
        return x
    
    def num_flat_features(self,x):
        size = x.size()[1:]
        num_features=1
        for s in size:
            num_features*=s
        return num_features
    
net=Net()

In [ ]:
#define Loss Function and Optimization method
criterion = nn.MSELoss()
optimizer = optim.SGD(net.parameters(),lr=lr,momentum=0.9)

In [ ]:
#load datasets from matlab
in_set_file = loadmat('DLCB_Dataset/DLCB_input.mat')
in_set = in_set_file['DL_input']#in_set.shape=(54481,256)
out_set_file = loadmat('DLCB_Dataset/DLCB_output.mat')
out_set = out_set_file['DL_output']#out_set.shape=(54481,2048)

In [ ]:
#check size
num_total_user=in_set.shape[0]
print('Number of total users in input dataset is '+str(num_total_user))
size_list_pr=[.001,.8]#[.001,.05,.1,.15,.2,.25,.3,.4,.45,.5,.55,.6,.65,.7,.75,.8,.85,.9,.95,1]
size_list=[]
for idx in range(len(size_list_pr)):
    size_list.append(int(size_list_pr[idx]*num_total_user))
print(size_list)
#merge two lists one by one
def mergeList(lst1,lst2):
    return np.array([[i,j] for i,j in zip(lst1,lst2)]).ravel()

In [ ]:
class DataSetGenerator(data.Dataset):
    def __init__(self,input_set,output_set):#input_set.shape= (43,256); output_set.shape=(43,2048)
        self.input_set = input_set
        self.output_set = output_set
        real = {}
        imag = {}
        self.order = {}
        for idx in range(len(input_set)): 
            real[idx] = np.float32(input_set[idx].real)
            imag[idx] = np.float32(input_set[idx].imag)
            self.order[idx] = mergeList(real[idx],imag[idx])
            
    def __getitem__(self,index):
        for idx in range(len(self.input_set)):
            x = self.order[idx]
            target = self.output_set[idx]
        return x,target
        
    def __len__(self):
        return len(self.input_set)

In [ ]:
#train the network
def train():
    for idx_size in range(len(size_list)):
    
        #assign values to set in_train, out_train, in_test, out_test
        size = size_list[idx_size]
        num_train = int(size*0.8)
        num_test = int(num_total_user*0.2)
        train_index = np.random.choice(range(0,num_total_user),size=num_train,replace=False)
        rem_index = set(range(0,num_total_user))-set(train_index)
        test_index = list(set(np.random.choice(list(rem_index),size=num_test,replace=False)))
        in_train = in_set[train_index]
        in_test = in_set[test_index]
        out_train = out_set[train_index]
        out_test = out_set[test_index]
        print('Scene ['+str(1+idx_size)+']/['+str(len(size_list))+']. '+str(size)+' MUs are served now.')
        print(str(num_train)+' training examples. '+str(num_test)+' test examples.')
    
        #divide every in_train, out_train example into real part and imaginary part
        train_generator = DataSetGenerator(input_set = in_train, output_set = out_train)
        test_generator = DataSetGenerator(input_set = in_test, output_set = out_test)
        train_loader = data.DataLoader(dataset=train_generator, batch_size=100,shuffle=True)
        test_loader = data.DataLoader(dataset=test_generator, batch_size=100,shuffle=True)
    
        for epoch in range(max_epoch):
            run_loss=0
            total_loss=[]
        
            for batch_idx,(x,target) in enumerate(train_loader):
                y = net(x)
                #calculate loss function
                loss = criterion(target,y)
                total_loss.append(loss)
                #zero parameter gradients
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
            
                #print loss statistics
                if batch_idx%100==99:
                    print('Train Epoch {}\t Step {}\t Loss {:.8f}'.format(epoch+1,batch_idx+1,sum(total_loss)/len(total_loss)),flush=True)
    print('End of Training.')
train()

In [ ]:
#save model
state = {'model':net.state_dict(),'optimizer':optimizer.state_dict(),'epoch':epoch}
path = './model.pth'
torch.save(state,path)

In [ ]:
#test the network
def test():
    for idx_size in range(len(size_list)):
        for epoch in range(max_epoch):
            test_loss = 0
            total_loss = []
            with torch.no_grad():
                for batch_idx,(x,target) in enumerate(test_loader):
                    y_hat = net(x)
                    test_loss = criterion(target,y_hat)
                    total_loss.append(test_loss)
                    optimizer.zero_grad()
                    loss.backward()
                    optimizer.step()
                    print('Test set: Average loss:{.8f}, Accuracy:{}/{}({:.2f})').format(test_loss,sum(total_loss),len(total_loss),100.*test_loss/len(total_loss))
    print('End of Testing.')
test()